In [97]:
from facebook_scraper import *
import pandas as pd
import time
import numpy as np
import datetime
from itertools import groupby
from time import sleep

In [98]:
cols = np.load('Data/Final/_cols.npy', allow_pickle=True)
arr = np.load('Data/Final/.npy', allow_pickle=True)
data_df = pd.DataFrame(arr, columns=cols)
data_df = data_df[['post_id', 'time', 'post_text', 
            'images_description', 'shares', 'comments',
                'comments_full', 'reaction_count',
                'reactions', 'reactors']]
data_df[['shares', 'comments', 'reaction_count']] = data_df[['shares', 'comments', 'reaction_count']].astype(int)

In [99]:
comments = pd.DataFrame(data_df[['post_id', 'time', 'comments', 'comments_full', 'shares']])

In [100]:
data_df['comments_full'] = data_df['comments_full'].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [101]:
cmt_text = ""
id_cmters = []
cmts = np.array(data_df['comments_full'])

def get(arr, cmt_txt=""):
    for cmt in arr:
        # print(cmt.keys())
        if cmt['commenter_id'] not in id_cmters:
            id_cmters.append(cmt['commenter_id'])
        cmt_txt += cmt['comment_text'] + " "
        if 'replies' in cmt.keys():
            cmt_txt += get(cmt['replies'])
    return cmt_txt

for cmt in cmts:
    cmt_text += get(cmt)
    # get(cmt)
    

In [102]:
len(id_cmters)

7728

In [103]:
user_profiles = []

In [104]:
max, index = 0, 0
user_profiles = []
cookie_path = 'www.facebook.com_cookies.txt'
for i in range(len(id_cmters)):
    try:
        profile = get_profile(id_cmters[i])
        
     
        if profile:
            keys_count = len(profile.keys())
            
          
            if len(profile.keys()) > max:
              max = len(profile.keys())
              index = i
            
            user_profiles.append(profile)
            print(profile)
        
    except Exception as e:
        print(f"Error processing user {id_cmters[i]}: {e}")
        continue


{'Friend_count': None, 'Follower_count': None, 'Following_count': None, 'cover_photo': 'https://scontent.fhan2-3.fna.fbcdn.net/v/t39.30808-6/385770945_1988434674853829_3693717170146559732_n.jpg?stp=cp0_dst-jpg_e15_fr_q65&_nc_cat=109&ccb=1-7&_nc_sid=ff2d77&_nc_ohc=-A5o83Fqt3EAX_fDbhj&_nc_ht=scontent.fhan2-3.fna&oh=00_AfBwBYu6WGhMd2qkKuBFB6XLgtzA_mzjM0yV1e5epJTgpA&oe=6572CEE7', 'profile_picture': 'https://scontent.fhan2-4.fna.fbcdn.net/v/t39.30808-1/306357363_1731502533880379_1758886671249344921_n.jpg?stp=cp1_dst-jpg_e15_p120x120_q65&_nc_cat=100&ccb=1-7&_nc_sid=4da83f&_nc_ohc=u14Yy3qE49kAX-QzFW4&_nc_ht=scontent.fhan2-4.fna&oh=00_AfA87RYURXL_dtlsWuJkBpezMI1EY9KD_WgGKxEFFbAJRA&oe=6572D123', 'id': '100010619473461', 'Name': 'Nam Trần', 'Nơi từng sống': 'Huế\nTỉnh/Thành phố hiện tại\nThuy Duong, Binh Tri Thien, Vietnam\nQuê quán'}
Error processing user 100005618973021: A login (cookies) is required to see this page
Error processing user 100015762163863: A login (cookies) is required to see t

KeyboardInterrupt: 

In [ ]:
if len(user_profiles) > 0:
    post_df_full = pd.DataFrame(user_profiles)
    path = 'Data/Final/'
    post_df_full.to_csv(path + "User" + '.csv', index=False)